In [31]:

!sudo apt install sqlite3
!cp $SQLITE_STORE_DIR/faiss_store.db .
!ls -lh faiss_store.db

counter = !cat counter.txt
counter = int(counter[0]) - 1
%env MEMORY_BANK_NAME=bank_pdf_paper_{counter}


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
sqlite3 is already the newest version (3.37.2-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
-rw-r--r-- 1 ubuntu ubuntu 52M Dec  7 04:45 faiss_store.db
env: MEMORY_BANK_NAME=bank_pdf_paper_23


In [36]:
%%bash
echo "select value from kvstore where key = 'faiss_index:v1::${MEMORY_BANK_NAME}';" \
  | sqlite3 faiss_store.db \
  > faiss_index.json


Runtime error near line 1: database disk image is malformed (11)


CalledProcessError: Command 'b'echo "select value from kvstore where key = \'faiss_index:v1::${MEMORY_BANK_NAME}\';" \\\n  | sqlite3 faiss_store.db \\\n  > faiss_index.json\n'' returned non-zero exit status 1.